Each row represnts characteristic of a single employee . Many categorical data has 
been coded to mask the data, you dont need to worry about their exact meaning 

satisfaction_level: satisfaction rating on scale of 0 to 1
last_evaluation: Evaluation rating on scale of 0 to 1
number_project: Number of projects employee is involved with
average_montly_hours: Average monthly hrs clocked in last quarte
time_spend_company: Time spent in the company ( in years)
Work_accident: Work Accident flag
left: Whether that employee has quit or not ( 1= Yes , 0 = No). This is the target variable
promotion_last_5years: Whether the employee was promoted in last 5 years (1=Yes, 0=No)
sales: Which sales category was his role in, the category sales represents generic sales profile
salary: Category of the salary

In [96]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, metrics
import statistics
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.ensemble import RandomForestClassifier

In [72]:
df_train = pd.read_csv("hr_train.csv")
df_test = pd.read_csv("hr_test.csv")

In [73]:
df_train.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.42,0.46,2,150,3,0,1,0,sales,medium
1,0.66,0.77,2,171,2,0,0,0,technical,medium
2,0.55,0.49,5,240,3,0,0,0,technical,high
3,0.22,0.88,4,213,3,1,0,0,technical,medium
4,0.20,0.72,6,224,4,0,1,0,technical,medium


In [74]:
## Serendipity, no missing values

df_test.isnull().sum()

satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
promotion_last_5years    0
sales                    0
salary                   0
dtype: int64

In [75]:
df_train.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000
mean,0.612683,0.717131,3.808553,201.059815,3.494238,0.144299,0.292885,0.021716
std,0.248578,0.171483,1.230572,49.959332,1.453227,0.351410,0.455108,0.145763
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [76]:
df_train["salary"].value_counts()

low       5104
medium    4515
high       880
Name: salary, dtype: int64

In [77]:
df_train["sales"].value_counts()

sales          2935
technical      1890
support        1556
IT              825
product_mng     639
marketing       614
hr              532
RandD           530
accounting      527
management      451
Name: sales, dtype: int64

In [78]:
df_train.shape

(10499, 10)

In [79]:
dummy_cols = ['sales', 'salary']

df_train = pd.get_dummies(df_train, columns=dummy_cols)
df_test = pd.get_dummies(df_test, columns=dummy_cols)

In [80]:
## Removing one dummy column for each of the categorical variable: Type, Method and CouncilArea

df_train = df_train.drop('sales_IT', axis = 1)
df_train = df_train.drop('salary_high', axis = 1)

df_test = df_test.drop('sales_IT', axis = 1)
df_test = df_test.drop('salary_high', axis = 1)

In [81]:
df_test.shape

(4500, 18)

In [82]:
df_train.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'sales_RandD', 'sales_accounting', 'sales_hr',
       'sales_management', 'sales_marketing', 'sales_product_mng',
       'sales_sales', 'sales_support', 'sales_technical', 'salary_low',
       'salary_medium'],
      dtype='object')

In [83]:
X_train = df_train[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_RandD', 'sales_accounting',
       'sales_hr', 'sales_management', 'sales_marketing', 'sales_product_mng',
       'sales_sales', 'sales_support', 'sales_technical',
       'salary_low', 'salary_medium']]

y_train = df_train["left"]

X_test = df_test

In [84]:
# the independent variables set
X = X_train
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [vif(X.values, i)
                          for i in range(len(X.columns))]
  
# print(vif_data)

## Here last_eval has maximum VIF of 21.48 so dropping it in both train and test

X = X.drop('last_evaluation', axis = 1)



# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [vif(X.values, i)
                          for i in range(len(X.columns))]
  
# print(vif_data)


## Here average_monthly_hours with max VIf = 18, so we drop it 
X = X.drop('average_montly_hours', axis = 1)
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [vif(X.values, i)
                          for i in range(len(X.columns))]
  
# print(vif_data)

## Next number project with VIf = 5, so we drop it
X = X.drop('number_project', axis = 1)

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [vif(X.values, i)
                          for i in range(len(X.columns))]
  
# print(vif_data)


## Removing satisfaction level VIF = 5.732

X = X.drop('satisfaction_level', axis = 1)

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [vif(X.values, i)
                          for i in range(len(X.columns))]
  
# print(vif_data)


## Removing time_spend_company VIF = 5.54


X = X.drop('time_spend_company', axis = 1)

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [vif(X.values, i)
                          for i in range(len(X.columns))]
  
# print(vif_data)

X_train = X

X_train.columns


Index(['Work_accident', 'promotion_last_5years', 'sales_RandD',
       'sales_accounting', 'sales_hr', 'sales_management', 'sales_marketing',
       'sales_product_mng', 'sales_sales', 'sales_support', 'sales_technical',
       'salary_low', 'salary_medium'],
      dtype='object')

In [85]:
# the independent variables set
X1 = X_test
  

X1 = X1.drop('last_evaluation', axis = 1)
X1 = X1.drop('average_montly_hours', axis = 1)
X1 = X1.drop('number_project', axis = 1)
X1 = X1.drop('satisfaction_level', axis = 1)
X1 = X1.drop('time_spend_company', axis = 1)

X_test = X1

X_test.columns

Index(['Work_accident', 'promotion_last_5years', 'sales_RandD',
       'sales_accounting', 'sales_hr', 'sales_management', 'sales_marketing',
       'sales_product_mng', 'sales_sales', 'sales_support', 'sales_technical',
       'salary_low', 'salary_medium'],
      dtype='object')

In [147]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)

In [148]:
# import the metrics class
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_train, y_pred)
cnf_matrix

array([[7424,    0],
       [3075,    0]], dtype=int64)

In [149]:
#use model to predict probability that given y value is 1
y_pred_proba = logreg.predict_proba(X_train)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_train, y_pred_proba)

#print AUC score
print(auc)

0.6074004984931315


In [108]:
y_train

0        1
1        0
2        0
3        0
4        1
        ..
10494    0
10495    0
10496    1
10497    1
10498    0
Name: left, Length: 10499, dtype: int64

Now trying to do with SVM

In [169]:
from sklearn.svm import SVC
model = SVC(probability=True)

In [170]:
model.fit(X_train, y_train)

SVC(probability=True)

In [171]:
y_pred = model.predict(X_train)

In [172]:
y_pred = model.predict(X_train)
cnf_matrix = metrics.confusion_matrix(y_train, y_pred)
cnf_matrix

array([[7423,    1],
       [3064,   11]], dtype=int64)

In [173]:
#use model to predict probability that given y value is 1
y_pred_proba = model.predict_proba(X_train)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_train, y_pred_proba)

#print AUC score
print(auc)


0.44476961119287917


In [142]:
X_train

,Work_accident,promotion_last_5years,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_low,salary_medium
0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10494,0,0,0,0,0,0,0,0,0,0,1,1,0
10495,0,0,0,0,0,0,1,0,0,0,0,0,0
10496,0,0,0,0,0,0,0,0,1,0,0,1,0
10497,0,0,0,0,0,0,0,0,0,1,0,1,0


In [166]:
forest = RandomForestClassifier(criterion='gini', 
                                 max_depth= 5,
                                 n_estimators=100,
                                 random_state=1,
                                 n_jobs=2)
#
# Fit the model
#
forest.fit(X_train, y_train)
 

RandomForestClassifier(max_depth=5, n_jobs=2, random_state=1)

In [167]:
#use model to predict probability that given y value is 1
y_pred_proba = forest.predict_proba(X_train)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_train, y_pred_proba)

#print AUC score
print(auc)


0.6200244209069246


In [168]:
y_pred = forest.predict(X_train)
cnf_matrix = metrics.confusion_matrix(y_train, y_pred)
cnf_matrix

array([[7424,    0],
       [3073,    2]], dtype=int64)

In [176]:
rf_pred_hr = pd.DataFrame(model.predict(X_test))
rf_pred_hr.to_csv("Shariya_Hasan_P4_part2.csv")


In [177]:
182/(182+250)

0.4212962962962963